In [1]:
import xarray as xr
import numpy as np
import metpy
import metpy.calc as mpycalc
from metpy.units import units
from wetbulb_dj08_spedup import WetBulb
import time
import glob
import pandas as pd

/home/estudiantes/anaconda/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Archivos de presión modelo GFDL
psl_GFDL_histo=glob.glob("/home/andreagvc/data/CMIP/psl_day_GFDL-CM4_historical_r1i1p1f1_gr2_*.nc") #Cambiar si es el otro modelo
psl_GFDL_ssp585=glob.glob("/home/andreagvc/data/CMIP/psl_day_GFDL-CM4_ssp585_r1i1p1f1_gr2_*.nc")

#Combinando todos los archivos
comb_psl_GFDL_histo= xr.open_mfdataset(psl_GFDL_histo, combine='by_coords')
comb_psl_GFDL_ssp585= xr.open_mfdataset(psl_GFDL_ssp585, combine='by_coords')

#Cortando las coordenadas a México
comb_psl_GFDL_histo=comb_psl_GFDL_histo.psl.where(comb_psl_GFDL_histo.psl.time.dt.year>1983,drop=True)
comb_psl_GFDL_histo=comb_psl_GFDL_histo.sel(lat=slice(5,37),lon=slice(360-125,360-75))

comb_psl_GFDL_ssp585=comb_psl_GFDL_ssp585.psl.where(comb_psl_GFDL_ssp585.psl.time.dt.year>2069,drop=True)
comb_psl_GFDL_ssp585=comb_psl_GFDL_ssp585.sel(lat=slice(5,37),lon=slice(360-125,360-75))

In [3]:
#Cargando los archivos de humedad y temperatura
#HadGEM
HadGEM_histo=xr.open_dataset("/home/andreagvc/data/CMIP/Results/cal_historical_HADGEM.nc")
HadGEM_ssp585=xr.open_dataset("/home/andreagvc/data/CMIP/Results/cal_ssp585_HADGEM.nc")

#GFDL
GFDL_histo=xr.open_dataset("/home/andreagvc/data/CMIP/Results/cal_historical_GFDL.nc")
GFDL_ssp585=xr.open_dataset("/home/andreagvc/data/CMIP/Results/cal_ssp585_GFDL.nc")

#Seleccionando las variables

#Histórico, HADGEM
Had_TW_histo=HadGEM_histo.Tw
Had_T_histo=HadGEM_histo.T
Had_q_histo=HadGEM_histo.SH

#Histórico, GFDL
GFDL_TW_histo=GFDL_histo.Tw
GFDL_T_histo=GFDL_histo.T
GFDL_q_histo=GFDL_histo.SH

#ssp585, HADGEM
Had_TW_ssp585=HadGEM_ssp585.Tw
Had_T_ssp585=HadGEM_ssp585.T
Had_q_ssp585=HadGEM_ssp585.SH

#ssp585, GFDL
GFDL_TW_ssp585=GFDL_ssp585.Tw.where(GFDL_ssp585.Tw.time.dt.year>2069,drop=True)
GFDL_T_ssp585=GFDL_ssp585.T.where(GFDL_ssp585.T.time.dt.year>2069,drop=True)
GFDL_q_ssp585=GFDL_ssp585.SH.where(GFDL_ssp585.SH.time.dt.year>2069,drop=True)


In [4]:
#TW con qhisto y Tescenario
lats=comb_psl_GFDL_histo.lat 
lons=comb_psl_GFDL_histo.lon 
temp=GFDL_T_ssp585.mean("time")
hum=GFDL_q_histo.mean("time")
pres=comb_psl_GFDL_histo.mean("time")

In [85]:
#Calculando TW con qhisto y Tescenario
def TW(lats,lons,temp,hum,pres):
    
    Twb=np.zeros(temp.shape) #Haciendo un array para rellenar los datos, dado que la rutina dos es más rápida, se ocupará ese método para calcular TW
    
    for ilat,lat in enumerate(lats):
        #print(ilat)
        for ilon,lon in enumerate(lons):
            #print(ilon)
            press=pres.isel(lat=ilat,lon=ilon)
            t=temp.isel(lat=ilat,lon=ilon)
            q=hum.isel(lat=ilat,lon=ilon)
    		# checamos si esta coordenada tiene datos o no, si son nulos, no calculamos nada
            if press.isnull():
                continue
            else:
    			
                hs= q.values
                
                SpecificHumidity=float(hs)
                Temperature=float(t.values-273.15)
               # print(Temperature)
                Pressure=float(press.values)
                #print(Temperature,Humidity,Pressure)
                Twb[ilat,ilon]=WetBulb(Temperature,Pressure,SpecificHumidity,0)[0]+273.15
                
            
    return Twb

In [86]:
#Calculando TW con qhisto y Tescenario
T_W=TW(lats,lons,temp,hum,pres)

In [87]:
arrF = xr.DataArray(T_W-273.15, dims=("lat","lon"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos 
                   coords={ "lat": lats,
                "lon": lons},
                   attrs={"units": "K"})

arrF.to_netcdf("twf_qh1.nc") #Guardando el archivo

In [92]:
#TW con qescenario y Thistórica
lats2=comb_psl_GFDL_histo.lat 
lons2=comb_psl_GFDL_histo.lon 

temp2=GFDL_T_histo.mean("time")
hum2=GFDL_q_ssp585.mean("time")
pres2=comb_psl_GFDL_histo.mean("time")

In [93]:
T_W2=TW(lats2,lons2,temp2,hum2,pres2)

In [94]:
arrF2 = xr.DataArray(T_W2-273.15, dims=( "lat","lon"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos 
                   coords={"lat": lats2,
                "lon": lons2},
                   attrs={"units": "K"})

arrF2.to_netcdf("twf_qs1.nc")